Importações

In [2]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym
from math import *


Exemplo teste

In [200]:
# Define variables
x1, x2, x3 = sym.symbols('x1 x2 x3')

# Define functions 
f1 = 3*x1 - sym.cos(x2*x3) - 0.5
f2 = x1**2 - 81*((x2 + 0.1)**2) + sym.sin(x3) + 1.06
f3 = sym.E**(-x1*x2) + 20*x3 + (10*pi - 3)/3

F = np.transpose(np.array([f1,f2,f3]))

# Define the values and jacabian matrix
## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]), (x3,x[2])]))
    return np.array(Fx)

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3)])
    return np.array(Jmatrix)


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]), (x3,x[2])]))
        i += 1
    return np.array(Jx)



## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.


In [183]:
# solve with Newton's method
x = np.transpose([.1, .1, -0.1]) # initial guess
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

methodNewton(n, x, tolerance, maxIterations, F)


Começando com Xo =  [ 0.1  0.1 -0.1]
1  Iteração x= [ 0.49986967  0.01946685 -0.52152047] Norma:  0.5865670056112852
2  Iteração x= [ 0.50001424  0.00158859 -0.52355696] Norma:  0.017994451377116808
3  Iteração x= [ 5.00000113e-01  1.24447833e-05 -5.23598450e-01] Norma:  0.0015767557491808675
4  Iteração x= [ 5.00000000e-01  7.75785723e-10 -5.23598776e-01] Norma:  1.2448781083988536e-05
5  Iteração x= [ 5.00000000e-01 -1.37632501e-17 -5.23598776e-01] Norma:  7.760833298279343e-10


array([ 5.00000000e-01, -1.37632501e-17, -5.23598776e-01])

In [252]:
# solve with Broyden's method
x = np.transpose(np.array([.1, .1, -0.1], dtype=np.float64)) # initial guess
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

methodBroyden(n, x, tolerance, maxIterations, F)

# Ao = calcJ(J(F), x)
# v = calcF(F, x)
# A = np.linalg.inv(np.array(Ao, dtype=np.float64)) # use gaussian elimination
# s = np.dot(-A, v)
# x = x + s
# s

Começando com Xo =  [ 0.1  0.1 -0.1]
X1 =  [0.499869672926428 0.0194668485374181 -0.521520471935831]
2  Iteração x= [0.500006844255838 0.00880875900894540 -0.523145868361381] Norma:  8.73004675071813
3  Iteração x= [0.496825832739135 0.00104272313797116 -0.526573897267491] Norma:  0.19768323091724893
4  Iteração x= [0.495011488371965 -0.00453597506235492 -0.528549739290024] Norma:  0.1418391343720871
5  Iteração x= [0.497620389917214 -0.00480387690089538 -0.526055594992199] Norma:  0.09302990251449662
6  Iteração x= [0.507909742111223 0.00539942680906302 -0.515784536976182] Norma:  0.051410083042303656
7  Iteração x= [0.375760991929020 -0.121439994746923 -0.647749906779201] Norma:  0.25007975899072193
8  Iteração x= [-4.85961970606986 -5.45823718438040 -5.88383763918513] Norma:  2.695598414950048
9  Iteração x= [11302.2173894133 11225.4979066584 11300.9724811458] Norma:  2303.5445308391227
10  Iteração x= [-95172324952.9105 -95483556512.6896 -95173967911.6328] Norma:  10079395501.04979

ValueError: array must not contain infs or NaNs

In [249]:
w = v
v = calcF(F, x)
y = v - w
z = np.dot(-A, y)
p = np.dot(-np.transpose(s), z)
ut = np.dot(np.transpose(s), A)
A = A + (1/p)*np.dot((s+z),ut)
s = np.dot(-A, v)
x = x + s
z

array([-0.399753513260174, 0.0698540502709545, 0.419874063847125],
      dtype=object)

In [250]:
#rascunho
np.dot(z+s, ut)

-0.0616314154575179

## ITEM 1

In [196]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
f1 = x1*x2 +x1 -3*x5
f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    
F = np.transpose(np.array([f1,f2,f3,f4,f5]))

In [197]:
## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
    return np.array(Fx)

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3), f.diff(x4), f.diff(x5)])
        
    return np.array(Jmatrix)    


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
        i += 1
        
        
    return np.array(Jx)

## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.



### Método de Newton

In [175]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    k = 1
    print("Começando com Xo = ", x)
    while k <= maxIterations:
        Fx = [-x for x in calcF(F,x)]
        Jx = calcJ(J(F), x)
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(Fx, np.float64))
        x = x + y
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            k += 1
        
    return output

In [176]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
methodNewton(n, Xo, tolerance, maxIterations, F)
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")


Começando com Xo =  [10 10 10 10 10]
1  Iteração x= [0.01065204 9.99922036 5.00102591 5.03641579 0.03645869] Norma:  15.769785630785213
2  Iteração x= [ 0.01001153 10.01012371  2.49920776  2.59112275  0.03674507] Norma:  3.498381268945265
3  Iteração x= [9.98373148e-03 1.00839478e+01 1.24338306e+00 1.43787284e+00
 3.68870704e-02] Norma:  1.7066139146402113
4  Iteração x= [9.68920699e-03 1.04317749e+01 6.06359646e-01 9.75892034e-01
 3.69557589e-02] Norma:  0.8603539169025427
5  Iteração x= [8.50591251e-03 1.18477258e+01 2.74174277e-01 8.66848855e-01
 3.69857062e-02] Norma:  1.4584772627650562
6  Iteração x= [5.41646124e-03 1.67133282e+01 1.03992448e-01 8.60022364e-01
 3.69918658e-02] Norma:  4.868583458334746
7  Iteração x= [2.92103470e-03 2.76408171e+01 6.02819450e-02 8.59724155e-01
 3.69765221e-02] Norma:  10.927576589262472
8  Iteração x= [3.45584277e-03 3.17170310e+01 6.96040198e-02 8.59516377e-01
 3.69616409e-02] Norma:  4.07622454194743
9  Iteração x= [3.43022114e-03 3.13294698e+0

### Método de Broyden

In [251]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F): # n is the number of equations and unknowns
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    Ao = calcJ(J(F), x)
    v = calcF(F, x)
    A = np.linalg.inv(np.array(Ao, dtype=np.float64)) # use gaussian elimination
    s = np.dot(-A, v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    while k <= maxIterations:
        w = v
        v = calcF(F, x)
        y = v - w
        z = np.dot(-A, y)
        p = np.dot(-np.transpose(s), z)
        ut = np.dot(np.transpose(s), A)
        A = A + (1/p)*np.dot((s+z),ut)
        s = np.dot(-A, v)
        x = x + s
        print(k," Iteração x=", x, end=" ")
        if np.linalg.norm(s, np.inf) < tolerance:
            print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break
        print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        k += 1
        
    return output
             
    

In [199]:
# solve with Broyden's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(np.array(x)) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-5) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
methodBroyden(n, Xo, tolerance, maxIterations, F)
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [10 10 10 10 10]
X1 =  [0.0106520416154137 9.99922036085834 5.00102590899330 5.03641579334684
 0.0364586887839735]
2  Iteração x= [-4.60411880907955 5.38639334438619 -0.862549628212813 -0.808783597588250
 -4.57689277646041] Norma:  1951.5790985841434
3  Iteração x= [-2.33690566579617 3.70976773411303 -1.57454022617500 -1.82063695981035
 -0.979156115056380] Norma:  626.2223544037204
4  Iteração x= [-0.179109795710622 4.15811909323119 -0.627361299042871 -1.13486473654929
 0.851498203634657] Norma:  142.4766810497612
5  Iteração x= [1.61434641754636 5.83165531500050 1.05866623012389 0.524877550861784
 1.72413381825492] Norma:  83.60305739502645
6  Iteração x= [0.344058639143646 6.03819946558486 0.854679428174380 0.500911404012575
 -0.0101863589179425] Norma:  45.28409973186194
7  Iteração x= [-0.506590841205134 5.52706490909668 0.220860092005079 -0.0536095494525800
 -0.541971979563440] Norma:  70.88701261156986
8  Iteração x= [-0.252419309268049 5.34164748312736 0.1354

### Comparação dos resultados

## ITEM 2

### Método de Newton

In [ ]:
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = .0005 # chosen
maxIterations = 100000 # so it doesnt loop forever in case of failure

def f(x):
    return


def methodNewton(p0, p1, tolerance, maxIterations):
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    i = 1
    q0 = f(p0)
    q1 = f(p1)
    while i <= maxIterations:
        p = p1 - q1*((p1-p0)/(q1-q0))
       
        if tolerance < p - p1 < tolerance:
            output = p
            break
        else:
            i += 1
            p0 = p1
            q0 = q1
            p1 = p
            q1 = f(p)
        
    return output

### Método de Broyden

In [ ]:
Po = x # initial approximation
tolerance = .0005 # arbitrary
maxIterations = 100000 # so it doesnt loop forever in case of failure
def f(x):
    return
def df(x):
    return
def J(x):
    result = df(x)
    return result
def F(x):
    return


def methodBroyden(n, x, tolerance, maxIterations): # n is the number of equations and unknowns
    output = "The method failed after "+maxIterations+" iterations. The procedure was unsuccessful"
    
    Ao = J(x)
    v = F(x)
    A = np.transpose(Ao) #use gaussian elimination
    s = -A * v
    x = x + s
    k = 2
    
    while k <= maxIterations:
        w = v
        v = F(x)
        y = v - w
        z = -A* y
        p = -np.transpose(s) * z
        ut = np.transpose(np.transpose(s) * A)
        A = A + (1/p)*(s+z)*ut
        s = -A * v
        x = x + s
        if np.linalg.norm(s) < tolerance:
            output = x
            break
        
        k += 1
        
    return output
          

### Comparação dos resultados

## ITEM 3

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10